### Testing loading of customized models in Compass

MIAPACA2 DTP/Parentl xml files

In [5]:
from Compass.compass import models, compass, utils
import importlib
import pandas as pd
import os
import cplex
import numpy as np
import libsbml

In [2]:
# Making changes in the local git repo - reload the .py file 
importlib.reload(models)
importlib.reload(models.importSBML3)
importlib.reload(models.geneSymbols)
importlib.reload(models.importCommon)

<module 'Compass.compass.models.importCommon' from '/home/dzhang/Compass_Integration/Compass/compass/models/importCommon.py'>

In [18]:
model_dir = os.path.abspath("Custom_xml_models/0_1_models/")
dtp_file_PATH = os.path.join(model_dir, "MIAPaca_DTP_0_75.xml")
par_file_PATH = os.path.join(model_dir, "MIAPaca_Par_0_75.xml")
HumanGEM_file_PATH = os.path.join(model_dir, "Human-GEM.xml")

In [6]:
HGDocument = libsbml.readSBMLFromFile(HumanGEM_file_PATH)

In [19]:
sbmlDocument = libsbml.readSBMLFromFile(dtp_file_PATH)

In [8]:
HGmodel = models.importSBML3.load("HumanGEM", HGDocument) 

In [21]:
model = models.importSBML3.load("HumanGEM", sbmlDocument) 

In [13]:
models.geneSymbols.resolve_genes(model)

In [14]:
model.reactions['MAR03905'].gene_associations.list_genes()

{'ADH-2',
 'ADH-3',
 'ADH-4',
 'ADH-5',
 'ADH1A',
 'ADH1B',
 'ADH1C',
 'ADH4',
 'ADH5',
 'ADH6',
 'ADH7',
 'ADHFE1',
 'ADHX',
 'FLJ32430',
 'GSNOR',
 'MGC45594',
 'ZADH2'}

In [16]:
models.importCommon.clean_reactions(model)

In [17]:
models.importCommon.limit_maximum_flux(model, 1000)

In [5]:
model = models.load_metabolic_model("HumanGEM", model_file_path = HumanGEM_file_PATH)

In [ ]:
# Load in model and initalize CPLEX problem 

model = models.init_model(model="HumanGEM", species="homo_sapiens", exchange_limit = 1.0, model_file_path='/home/dzhang/Compass_Integration/model_xml/0_1_models/MIAPaca_PAR_0_75.xml')

In [23]:
# Penalties.txt is for MIA PACA DTP 
reaction_penalties = pd.read_csv(
        '/home/dzhang/Compass_Integration/penalties.txt', sep="\t", header=0,
        usecols=[0, 1]) #0 is the Reaction column, only using 1st sampel 
reaction_penalties = reaction_penalties.set_index("Reaction").iloc[:, 0]
reaction_penalties

Reaction
MAR03905_pos    0.105728
MAR03907_pos    0.114925
MAR04097_pos    0.144783
MAR04099_pos    0.379900
MAR04108_pos    0.144783
                  ...   
MAR20045_pos    0.115123
MAR20052_pos    0.153752
MAR20066_pos    0.087496
MAR20069_pos    0.138128
MAR20070_pos    0.101348
Name: WT.d_S141_L007_R1_001, Length: 12494, dtype: float64

In [10]:
print(os.getcwd())

/home/dzhang/Compass_Integration


In [11]:
from Compass.compass.compass import algorithm

In [13]:
problem = algorithm.initialize_cplex_problem(model)

In [18]:
from Compass.compass import utils

In [20]:
import cplex

In [21]:
# calculating theoretical max flux works w/o error 
import tqdm
import sys
maxes = {}
reactions = list(model.reactions.values())

rxn_values = pd.DataFrame(columns=["Reaction ID", "Rxn Max", "Rxn Score"])
rxn_values = rxn_values.set_index("Reaction ID")

for reaction in reactions:
    
    if reaction.is_exchange:
        continue

    partner_reaction = reaction.reverse_reaction

    # Set partner reaction upper-limit to 0 in problem
    # Store old limit for later to restore
    if partner_reaction is not None:
        partner_id = partner_reaction.id
        old_partner_ub = problem.variables.get_upper_bounds(partner_id)
        old_partner_lb = problem.variables.get_lower_bounds(partner_id)
        problem.variables.set_upper_bounds(partner_id, max(old_partner_lb, 0))

    utils.reset_objective(problem)
    problem.objective.set_linear(reaction.id, 1.0)
    problem.objective.set_name(str(reaction.id))
    problem.objective.set_sense(problem.objective.sense.maximize)

    problem.solve()
    
    rxn_max = problem.solution.get_objective_value()
    maxes[reaction.id] = rxn_max
    
    if rxn_max != 0:
        problem.linear_constraints.add(
                lin_expr=[cplex.SparsePair(ind=[reaction.id], val=[1.0])],
                senses=['R'],
                rhs=[0.95 * rxn_max],
                names=['REACTION_OPT'])
        if(problem.objective.get_name() != 'reaction_penalties'):
                print(problem.objective.get_name())
                utils.reset_objective(problem)
                problem.objective.set_linear(
                    list(reaction_penalties.iteritems())
                )
                problem.objective.set_name('reaction_penalties')
                problem.objective.set_sense(problem.objective.sense.minimize)
        problem.solve()
        try: 
            value = problem.solution.get_objective_value()
            # Insert values 
            rxn_values.loc[reaction.id] = [value, rxn_max]
        except:
            rxn_values.loc[reaction.id] = [-1, rxn_max]
        problem.linear_constraints.delete('REACTION_OPT')
    
    if partner_reaction is not None:
        partner_id = partner_reaction.id
        problem.variables.set_upper_bounds(partner_id, old_partner_ub)

CplexSolverError: CPLEX Error  1016: Community Edition. Problem size limits exceeded. Purchase at http://ibm.biz/error1016.